# 05-2 교차 검증과 그리드 서치
검증 세트가 필요한 이유를 이해하고 교차 검증에 대해 배운다. 그리드 서치와 랜덤 서치를 이용해 최적의 성능을 내는 하이퍼파라미터를 찾는다.

## 검증 세트
테스트 세트를 사용해 성능을 확인하다 보면 점점 테스트 세트에 맞추게 되는 셈이다.

1. 훈련 세트에서 검증 세트를 나누어, 훈련 세트에서 모델을 훈련하고 검증 세트로 모델을 평가한다.
1. 훈련 세트와 검증 세트를 합쳐 전체 훈련 데이터에서 모델을 다시 훈련 

In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [2]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [4]:
# 훈련세트에서 검증 세트를 나눈다 (sub : 훈련, val : 검증)
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [5]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [6]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


## 교차 검증
검증 세트를 너무 조금 떼어 놓으면 검증 점수가 불안정할 것이다.

교차검증을 이용하면 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터를 사용할 수 있다. '교차검증'은 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복하고, 이 점수를 평균하여 최종 검증 점수를 얻는다.

- k-폴드 교차 검증 : 훈련 세트를 k 부분으로 나눠서 교차 검증 수행

In [7]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
# cross_validate : 먼저 평가할 모델 객체를 첫 번째 매개변수로 전달. 그 다음 훈련 세트 전체를 전달
# cross_validate() 함수는 기본적으로 5-교차 검증 수행

print(scores)

{'fit_time': array([0.00739193, 0.00707483, 0.00785327, 0.00759697, 0.00756931]), 'score_time': array([0.00078893, 0.00074983, 0.00081801, 0.00084209, 0.00073862]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [8]:
# 교차 검증의 최종 점수는 test_score 키에 담긴 5개의 점수를 평균하여 얻을 수 있음
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [9]:
# 교차 검증을 할 때 훈련 세트를 섞으려면 분할기(splitter)를 지정해야 한다
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
# 회귀 : KFold / 분류 : StratifiedKFold

print(np.mean(scores['test_score']))

0.855300214703487


In [10]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)

print(np.mean(scores['test_score']))

0.8574181117533719


## 하이퍼파라미터 튜닝
- 모델 파라미터 : 머신러닝 모델이 학습하는 파라미터
- 하이퍼 파라미터 : 사용자 지정 파라미터

그리드서치 : 사이킷런의 GridSearchCV 클래스는 하이퍼 파라미터 탐색과 교차 검증을 한번에 수행한다.

In [11]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001,0.0002,0.0003,0.0004,0.0005]}

In [12]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
# 탐색 대상 모델, params 변수, 병렬 실행에 사용할 CPU 코어 수 지정 (-1은 시스템에 있는 모든 코어 사용)

In [13]:
gs.fit(train_input, train_target)
# 사이킷런의 그리드 서치는 훈련이 끝나면 n개의 모델 중에서 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델을 훈련한다
# 이 모델은 gs 객체의 best_estimator_ 속성에 저장되어 있음
# 그리드 서치로 찾은 최적의 매개변수는 best_params_ 속성에 저장되어 있음

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [14]:
dt = gs.best_estimator_

print(dt.score(train_input, train_target))
print(gs.best_params_)

0.9615162593804117
{'min_impurity_decrease': 0.0001}


In [15]:
# 각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_results_ 속성의 'mean_test_score' 키에 저장되어 있음
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [16]:
# 넘파이 argmax() 함수를 사용하면 가장 큰 값의 인덱스를 추출할 수 있다.
# 그다음 이 인덱스를 사용해 params 키에 저장된 매개변수를 출력할 수 있음
best_index = np.argmax(gs.cv_results_['mean_test_score'])

print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


이 과정을 정리해보자
1. 먼저 탐색할 매개변수를 지정한다.
2. 그 다음 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾는다. 이 조합은 그리드 서치 객체에 저장된다.
3. 그리드 서치는 최상의 매개변수에서 (교차 검증에 사용한 훈련 세트가 아니라) 전체 훈련 세트를 사용해 최종 모델을 훈련한다. 이 모델도 그리드 서치 객체에 저장된다.

In [17]:
# 더 복잡한 매개변수 조합 탐색해보자
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5,20,1),
          'min_samples_split': range(2,100,10)
          }

In [18]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [19]:
# 최상의 매개변수 조합 확인
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [20]:
# 최상의 교차 검증 점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


### **랜덤 서치**

랜덤 서치에는 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달한다.

랜덤 서치에 randint과 uniform 클래스 객체를 넘겨주고 총 몇 번을 샘플링해서 최적의 매개변수를 찾으라고 명령할 수 있다.

In [21]:
from scipy.stats import uniform, randint

In [22]:
rgen = randint(0, 10)
rgen.rvs(10)

array([3, 6, 4, 5, 9, 8, 9, 1, 2, 3])

In [23]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 94,  98,  96, 111,  97,  94, 104, 105,  95, 106]))

In [24]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.13798177, 0.89878296, 0.08371601, 0.64292032, 0.63006452,
       0.51260026, 0.01991604, 0.27560139, 0.89579148, 0.92566537])

In [25]:
# 랜덤 서치에 randint과 uniform 클래스 객체를 넘겨주고 총 몇 번을 샘플링해서 최적의 매개변수를 찾으라고 명령할 수 있음.
# 탐색할 매개변수 범위
params = {'min_impurity_decrease':uniform(0.0001, 0.001),
          'max_depth':randint(20, 50),
          'min_samples_split':randint(2,25),
          'min_samples_leaf':randint(1,25)}

In [26]:
# 샘플링 횟수는 사이킷런의 랜덤 서치 클래스인 RandomizedSearchCV의 n_iter 매개변수에 지정
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fbb28b7a710>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fbb28b7ab10>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fbb29c54fd0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fbb28b6dad0>},
                   random_state=42)

In [27]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [28]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [29]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86


## 확인문제 3.
마지막 RandomizedSearchCV 예제에서 DecisionTreeClassifier 클래스에 splitter='random' 매개변수를 추가하고 다시 훈련해 보세요. splitter 매개변수의 기본값은 'best'로 각 노드에서 최선의 분할을 찾습니다. 'random'이면 무작위로 분할한 다음 가장 좋은 것을 고릅니다. 테스트 세트에서 성능이 어떻게 되었나요? (이런 옵션이 필요한 이유는 다음 절에서 알 수 있음)

In [30]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fbb28b7a710>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fbb28b7ab10>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fbb29c54fd0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fbb28b6dad0>},
                   random_state=42)

In [31]:
print(gs.best_params_)

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}


In [32]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8458726956392981


In [33]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))
# 성능 내려감 (결정 트리의 노드를 랜덤하게 분할하기 때문에, 100번의 반복에서 최적의 매개변수 조합을 찾지 못함)

0.786923076923077
